# Optimizing Hugging Face Models with Supervised Fine-Tuning (SFT)

NeMo 2.0 enables users to perform Supervised Fine-Tuning (SFT) and Parameter-Efficient Fine-Tuning (PEFT) using Hugging Face (HF) Large Language Models (LLMs). It utilizes HF's auto classes to download and load transformer models, and wraps these models as Lightning modules to execute tasks like SFT and PEFT. The goal of this feature is to provide day-0 support for the models available in HF.

[AutoModel](https://huggingface.co/docs/transformers/en/model_doc/auto) is the generic model class that is instantiated as one of the model classes from the library when created with the from_pretrained() class method. There are many AutoModel classes in HF, each covering a specific group of transformer model architectures. The AutoModel class primarily loads the base transformer model that converts embeddings to hidden states. For example, a specific AutoModel class like AutoModelForCausalLM includes a causal language modeling head on top of the base model.

NeMo 2.0 includes wrapper classes for these HF AutoModel classes, making them runnable in NeMo pretraining, SFT, and PEFT workflows by converting them into Lightning modules. Due to the large number of AutoModel classes, NeMo 2.0 currently includes only the widely used auto classes.

In this notebook, we will demonstrate a SFT training example on how to perform SFT with Hugging Face LLMs to make the models more performant on a specific task. We will focus on the models that can be loaded using the HF's `AutoModelForCausalLM` class.

<font color='red'>NOTE:</font> Due to the limitations of the Jupyter Notebook, the example in this notebook works only on a single GPU. However, if you move the code to a script, you can run it on multiple GPUs. If you are interested in running a multi-GPU example using the Jupyter Notebook, please check the SFT example in NeMo-Run.

## Step 1: Import Modules and Prepare the Dataset

In [ ]:
from functools import partial

import fiddle as fdl
import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import DataLoader

from nemo import lightning as nl
from nemo.collections import llm
from nemo.lightning.pytorch.callbacks import JitConfig, JitTransform

We will use the [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) dataset provided by NeMo 2.0, which is a reading comprehension dataset consisting of questions and answers pairs. The SquadDataModule extends the `FineTuningDataModule`, giving it access to existing data-handling logic, including packed sequences.

In [ ]:
class SquadDataModuleWithPthDataloader(llm.SquadDataModule):
    """Creates a squad dataset with a PT dataloader"""

    def _create_dataloader(self, dataset, mode, **kwargs) -> DataLoader:
        return DataLoader(
            dataset,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
            persistent_workers=self.persistent_workers,
            collate_fn=dataset.collate_fn,
            batch_size=self.micro_batch_size,
            **kwargs,
        )


def squad(tokenizer, mbs=1, gbs=2) -> pl.LightningDataModule:
    """Instantiates a SquadDataModuleWithPthDataloader and return it

    Args:
        tokenizer (AutoTokenizer): the tokenizer to use

    Returns:
        pl.LightningDataModule: the dataset to train with.
    """
    return SquadDataModuleWithPthDataloader(
        tokenizer=tokenizer,
        seq_length=512,
        micro_batch_size=mbs,
        global_batch_size=gbs,
        num_workers=0,
        dataset_kwargs={
            "sanity_check_dist_workers": False,
            "get_attention_mask_from_fusion": True,
        },
    )

## Step 2: Set Parameters and Start the SFT with a HF Model

Now, we will set some of the important variables, including the HF model name, maximum steps, number of GPUs, etc. You can find the details of these parameters below.
- `model_name`: Pre-trained HF model or path of a HF model.
- `strategy`: Distributed training strategy such as DDP, FSDP, etc. 
- `devices`: Number of GPUs to be used in the training.
- `max_steps`: Number of steps in the training.
- `wandb_project`: wandb project.
- `use_torch_jit`: Enable torch jit or not.
- `ckpt_folder`: Path for the checkpoins.

All popular models, including Llama, GPT, Gemma, Mistral, Phi, and Qwen, are supported. After running this workflow, please select another HF model and rerun the notebook with that model. Ensure the chosen model fits within your GPU(s) memory.

In [ ]:
# In order to use the models like Llama, Gemma, you need to ask for permission on the HF model page and then pass the HF_TOKEN in the next cell.
# model_name = "google/gemma-2b" # HF model name. This can be the path of the downloaded model as well.
model_name = "meta-llama/Llama-3.2-1B"  # HF model name. This can be the path of the downloaded model as well.
strategy = "auto" # Distributed training strategy such as DDP, FSDP2, etc.
max_steps = 100 # Number of steps in the training loop.
accelerator = "gpu"
wandb_project = None
use_torch_jit = False # torch jit can be enabled.
ckpt_folder="/opt/checkpoints/automodel_experiments/" # Path for saving the checkpoint.

Some models have gated access. If you are using one of those models, you will need to obtain access first. Then, set your HF Token by running the cell below.

In [ ]:
import os
os.environ["HF_TOKEN"] ='<HF_TOKEN>'

After setting some parameters, we can start the SFT training workflow. Although the SFT workflow with HF models/checkpoints differs slightly from workflows with NeMo models/checkpoints, we still use the same NeMo 2.0 API. The main difference is the model we pass into the `fine-tune` API.


In [ ]:
wandb = None
if wandb_project is not None:
    model = '_'.join(args.model.split('/')[-2:])
    wandb = WandbLogger(
        project=wandb_project,
        name=f'{model}_dev{devices}_strat_{strategy}',
    )

callbacks = []
if use_torch_jit:
    jit_config = JitConfig(use_torch=True, torch_kwargs={'dynamic': False}, use_thunder=False)
    callbacks = [JitTransform(jit_config)]

callbacks.append(
    nl.ModelCheckpoint(
        every_n_train_steps=max_steps // 2,
        dirpath=ckpt_folder,
    )
)

if strategy == 'fsdp2':
    astrategy = nl.FSDP2Strategy(data_parallel_size=devices, tensor_parallel_size=1)

llm.api.finetune(
    model=llm.HFAutoModelForCausalLM(model_name=model_name),
    data=squad(llm.HFAutoModelForCausalLM.configure_tokenizer(model_name), gbs=1),
    trainer=nl.Trainer(
        devices=1,
        max_steps=max_steps,
        accelerator="gpu",
        strategy=strategy,
        log_every_n_steps=1,
        limit_val_batches=0.0,
        num_sanity_val_steps=0,
        accumulate_grad_batches=1,
        gradient_clip_val=1.0,
        use_distributed_sampler=False,
        logger=wandb,
        callbacks=callbacks,
        precision="bf16",
    ),
    optim=fdl.build(llm.adam.pytorch_adam_with_flat_lr(lr=1e-5)),
    log=None,
)

## Step 3: Generate Output with the HF Pipeline

Once the SFT training is completed, you can generate outputs using HF's APIs to evaluate their quality. The fine-tuned model will be saved in a folder defined by the `ckpt_folder` variable. After the first run, the new checkpoint will be saved in a folder named `default--val_loss=0.0000-epoch=0-consumed_samples=0-last.ckpt`. If you run this notebook multiple times, you will see multiple checkpoints in the same location.

In [ ]:
import torch
import os
from pathlib import Path
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

new_checkpoint = Path(ckpt_folder) / "default--val_loss=0.0000-epoch=1-consumed_samples=0-last.ckpt"

pipe = pipeline(
    "text-generation",
    model=AutoModelForCausalLM.from_pretrained(new_checkpoint),
    tokenizer=AutoTokenizer.from_pretrained(new_checkpoint),
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

pipe("The key to life is")